In [1]:
using Pkg, Revise, Test
Pkg.activate(ENV["JULIA_DQMC"])
include(joinpath(ENV["JULIA_DQMC"], "src/dqmc_framework.jl"))


function mc_from_inxml(inxml::AbstractString)
    Random.seed!(1234)
    if SUPPRESS_DQMC_INIT_OUTPUT
        old_stdout = stdout
        (rp, wp) = redirect_stdout()
    end

    @assert isfile(inxml)
    p = Params()
    p.output_file = "$(inxml[1:end-7]).out.h5.running"
    xml2parameters!(p, inxml)
    mc = DQMC(p)
    init!(mc)

    if SUPPRESS_DQMC_INIT_OUTPUT
        redirect_stdout(old_stdout)
        close(wp); close(rp);
    end
    return mc
end

function mc_from_inxml(inxml::AbstractString, initconf)
    mc = mc_from_inxml(inxml)
    mc.p.hsfield .= initconf
    return mc
end

(@isdefined SUPPRESS_DQMC_INIT_OUTPUT) || (const SUPPRESS_DQMC_INIT_OUTPUT = false)
(@isdefined randconf) || (global const randconf = load("data/O3.jld", "randconf"));

# Run all tests

In [2]:
# const SUPPRESS_DQMC_INIT_OUTPUT = true
include("runtests.jl")

Running tests on thinkable.
L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1


┌ Warning: Calling somewhat inefficient and potentially inaccurate `inv_one_plus_udt`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:170
┌ Warning: Calling potentially inaccurate `inv_one_plus_udt!`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:204
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: UDT_to_mat! probably shouldn't be called here
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\linalg.jl:604
┌ Warning: Calling somewhat inefficient and potentially inaccurate `inv_sum_udts`
└ @ Main C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\

L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1
L = 5
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_

# Edit area

In [88]:
includet(joinpath(ENV["JULIA_DQMC"], "src/fermion_measurements.jl"))

# Run subset of tests

In [2]:
mc = mc_from_inxml("parameters/O3_generic_small_system.in.xml", randconf)
# mc_odd_L = mc_from_inxml("parameters/O3_odd_L_small_system.in.xml", load("data/O3.jld", "randconf_odd_L"))
# mc_nob = mc_from_inxml("parameters/O3_no_bfield_small_system.in.xml", randconf)
# mc_nob_nochkr = mc_from_inxml("parameters/O3_no_bfield_no_chkr_small_system.in.xml", randconf);

L = 4
SLICES = 10
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 11 -1


In [64]:
include("tests_freefermions.jl")

L = 2
SLICES = 80
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 1
CHECKERBOARD = FALSE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.0
U = 0.0
R = 1.0
C = 0.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = FALSE
WRITE_EVERY_NTH = 10
OPDIM = 1

CB = CBFalse
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:/Users/carsten/Desktop/sciebo/lattices/square_L_2_W_2.xml"


Loading lattice with 4 sites
Initializing neighbor-tables
Initializing hopping exponentials

Initializing HS field
Initializing boson action

Building stack
Initial propagate: 81 -1
Performing real-space ED...
Done with ED.
Allocated memory for ETPC measurements.
Allocated memory for ETCDC measurements.
Test Summary:                   | Pass  Total
Compare to real-space ED (free) |    7      7
L = 8
SLICES = 100
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 400
SAFE_MULT = 10
CHECKERBOARD = FALSE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.0
U = 0.0
R = 1.0
C = 0.0
GLOB